# Step1: Use Whisper API to transcribe a mp3 downloaded from a youtube
# Step2: Use Chat API to summarize a long-word transcript

### Download a youtube and convert it to mp3

In [1]:
# Before the following code, please install ffmepg: https://www.gyan.dev/ffmpeg/builds/
import sys
import yt_dlp

#Setting option
ydl_opts = {
            "format":"bestaudio/best",
            "outtmpl":"joeman.%(ext)s", # "%(title)s.%(ext)s"
            'ffmpeg_location': r'C:\ffmpeg-6.0-essentials_build\bin',
            "postprocessors":[{
                            "key":"FFmpegExtractAudio",
                            "preferredcodec":"mp3",
                            "preferredquality":"192",
            }]
}

# the url if yt
_id = "PRp3kJtMQwU"
url = "https://www.youtube.com/watch?v=" + _id

# build yt_dlp下載器物件
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=PRp3kJtMQwU
[youtube] PRp3kJtMQwU: Downloading webpage
[youtube] PRp3kJtMQwU: Downloading android player API JSON
[info] PRp3kJtMQwU: Downloading 1 format(s): 251
[dashsegments] Total fragments: 3
[download] Destination: joeman.webm
[download] 100% of   22.96MiB in 00:00:02 at 11.09MiB/s              
[ExtractAudio] Destination: joeman.mp3
Deleting original file joeman.webm (pass -k to keep)


### Divide the video into several segments

In [2]:
# note that ffmpeg.exe & ffprobe.exe & should exist in the current directory
from pydub import AudioSegment


# load mp3 file
sound = AudioSegment.from_file("joeman.mp3",format = 'mp3')

# set the length of each segment (unit: milisecond)
segment_length = 500000 # ~ 8.3 min/file

# split file
# Finally, there are three files created
for i, chunk in enumerate(sound[::segment_length]):
    # set the filename
    chunk.export(f"output_{i}.mp3",format="mp3")

### Use Whisper API

In [5]:
import os
import openai

# API Key
openai.api_key = "Y"
audio_file = open("output_0.mp3",'rb')

# Transcription API
# Note that the maximum content size is 26214400 bytes
# whisper-1 is a speech recognition model
transcript_0 = openai.Audio.transcribe("whisper-1",audio_file)
completed_text_0 = transcript_0.to_dict().get("text")
completed_text_0

"金融世界的風險真的很大 我花過130萬整修一間套房 因為那個韓國學生 他把裡面弄到 像第二次世界大戰之後被摧毀的廢墟 你真的想不到 租金應該5、6萬就不錯了吧 8萬 8萬 那你投保你10% 所以足以吃掉你的房價漲幅的一大部分 就算你很會買 跟你講 你如果沒有會看這個 你就死定了 每年要多付420萬日幣的稅 也就是一年100多 大家好 我是Joeman 那我們這一系列的東京買房企劃 這一集非常值得一看 因為我們邀請到 聽說在日本買房地產已經有十幾年 經歷的這個老師 竟然跟我們分享他的心得 就是我們的淡如姐 淡如姐你好 你好 大家好 不過這個會有人看嗎 會啦 因為這是小眾 有錢到去日本要去買房的人 都是要中產以上 這個節目一部分是要讓那些 不想在台灣買房的人 意淫用的 就是說 憑什麼新北或是台中 或新竹 意淫這兩個字用的挺好 對 就是 一樣是一坪4、50萬 我為什麼不買東京二三區 但問題是你人在台北 對 因為很常在房地產的論戰當中 大家就說什麼 青埔拿值那個價 竹北拿值那個價 可是你工作就在那裡 對啊 不過先來問一下淡如姐 我查資料是你 2010年就開始進軍日本房地產 我小孩09年出生 而且那時候剛好是金融風暴 慢慢的在結束 其實那時候已經經過了兩年 蠻悲哀的時候 我生小孩是生到快死了 你知道人快死的時候 不會關心你的錢 那時候我就發現說 我的確是有一筆存款 我放在澳幣裡面 最糟的時候 我損失了30% 但因為那時候 我家小孩有一整年 還有包括我自己 都在快沒命的狀態 所以我沒有去看他 不理有一種好處 本來虧了30% 突然他長了 就終於回來了 讓我意識到說 金融世界的風險真的很大 那索性回來了 是不是要換實在的具體的東西 當時我也覺得說 差不多可以退休了 其實那時候我對於整個 我的工作環境有點灰心 因為要講八卦才能夠 電視有收視率 然後不管做什麼 大家都在一窩蜂吵 直到把你吵死為止 做寶物鑑定也是這樣 我累了 我也知道電視是鐵達尼號 他果然慢慢往下沉 這也是對的 那時候我被記者干擾的很嚴重 所以你是覺得在台灣買 太高調嗎 也不是 其實台灣買有很多好處 我那時候如果在台灣買 恐怕也富可敵國 也是 因為08、09年 台灣的房地產也是跌的 也沒有很好 對 也不好 那時候我就 其實我有點灰心 我有點想去日本生活 是想要長定居在日本 後來我也有拿到日本的長居 可是我放棄了 很少人

### Use Chat API to do summary

In [14]:
# Chat API
def send_respond_gpt_turbo(transcript):
    response = openai.ChatCompletion.create(
                                    model = "gpt-3.5-turbo",
                                    messages=[
                                        {"role":"system","content":"請你成為文章摘要的小幫手，摘要以下文字，以繁體中文輸出"},
                                        {"role":"user","content":transcript}
                                    ],
                                    n=1, # num of ans
                                    max_tokens=2048 # num of tokens, input+output, 4096 is max
                                )
    complete_text=response.choices[0].message.content
    return complete_text

In [10]:
# Concatenate all transcripts
audio_file = open("output_1.mp3",'rb')
transcript_1 = openai.Audio.transcribe("whisper-1",audio_file)
completed_text_1 = transcript_1.to_dict().get("text")
audio_file = open("output_2.mp3",'rb')
transcript_2 = openai.Audio.transcribe("whisper-1",audio_file)
completed_text_2 = transcript_2.to_dict().get("text")

transcript = completed_text_0 + " " + completed_text_1 + " " + completed_text_2
transcript

"金融世界的風險真的很大 我花過130萬整修一間套房 因為那個韓國學生 他把裡面弄到 像第二次世界大戰之後被摧毀的廢墟 你真的想不到 租金應該5、6萬就不錯了吧 8萬 8萬 那你投保你10% 所以足以吃掉你的房價漲幅的一大部分 就算你很會買 跟你講 你如果沒有會看這個 你就死定了 每年要多付420萬日幣的稅 也就是一年100多 大家好 我是Joeman 那我們這一系列的東京買房企劃 這一集非常值得一看 因為我們邀請到 聽說在日本買房地產已經有十幾年 經歷的這個老師 竟然跟我們分享他的心得 就是我們的淡如姐 淡如姐你好 你好 大家好 不過這個會有人看嗎 會啦 因為這是小眾 有錢到去日本要去買房的人 都是要中產以上 這個節目一部分是要讓那些 不想在台灣買房的人 意淫用的 就是說 憑什麼新北或是台中 或新竹 意淫這兩個字用的挺好 對 就是 一樣是一坪4、50萬 我為什麼不買東京二三區 但問題是你人在台北 對 因為很常在房地產的論戰當中 大家就說什麼 青埔拿值那個價 竹北拿值那個價 可是你工作就在那裡 對啊 不過先來問一下淡如姐 我查資料是你 2010年就開始進軍日本房地產 我小孩09年出生 而且那時候剛好是金融風暴 慢慢的在結束 其實那時候已經經過了兩年 蠻悲哀的時候 我生小孩是生到快死了 你知道人快死的時候 不會關心你的錢 那時候我就發現說 我的確是有一筆存款 我放在澳幣裡面 最糟的時候 我損失了30% 但因為那時候 我家小孩有一整年 還有包括我自己 都在快沒命的狀態 所以我沒有去看他 不理有一種好處 本來虧了30% 突然他長了 就終於回來了 讓我意識到說 金融世界的風險真的很大 那索性回來了 是不是要換實在的具體的東西 當時我也覺得說 差不多可以退休了 其實那時候我對於整個 我的工作環境有點灰心 因為要講八卦才能夠 電視有收視率 然後不管做什麼 大家都在一窩蜂吵 直到把你吵死為止 做寶物鑑定也是這樣 我累了 我也知道電視是鐵達尼號 他果然慢慢往下沉 這也是對的 那時候我被記者干擾的很嚴重 所以你是覺得在台灣買 太高調嗎 也不是 其實台灣買有很多好處 我那時候如果在台灣買 恐怕也富可敵國 也是 因為08、09年 台灣的房地產也是跌的 也沒有很好 對 也不好 那時候我就 其實我有點灰心 我有點想去日本生活 是想要長定居在日本 後來我也有拿到日本的長居 可是我放棄了 很少人

In [11]:
# Split transcript into transcript array
transcript_arr = []
ret = ""
for script in transcript.split():
    ret = ret + " " + script
    if len(ret) > 1000:
        transcript_arr.append(ret)
        ret = ""
transcript_arr.append(ret)
print(f"the original atricle becomes {len(transcript_arr)} segments")

the original atricle becomes 9 segments


In [17]:
# Summarize transcript by parts
result_arr = []
for _i, _t in enumerate(transcript_arr):
    _text = send_respond_gpt_turbo(_t)
    result_arr.append(_text)
    print(f"Summary of Segment-{_i+1}:")
    print(_text)

Summary of Segment-1:
這是一則關於在日本買房地產的節目，主持人邀請了一位在日本投資房地產十幾年的老師淡如姐分享經驗。淡如姐表示金融風險真的很大，自己當時曾損失30%的存款。她認為在台灣買房有很多好處，但當時因為工作環境厭倦，加上想在日本長居，所以開始在日本投資房地產。但也提到在日本買房有風險，租金高達8萬，十分龐大，加上每年有420萬日圓的稅需要繳納。節目旨在提供給有意在日本投資的中產階級參考。
Summary of Segment-2:
此篇文章談論到作者在日本投資房地產的經驗，因公司需要而開始關心日本房地產。然而，由於日本的稅賦重，尤其是在每年的租金收入上，若不買賣房產，就要繳納較高的稅金。作者表示貸款金額較高，而且自己也不可能第一次買就買價值達兩億到三億日幣的物件。最後作者分享自己購買居酒屋的經驗。
Summary of Segment-3:
本文介紹東京大學附近的文教區，建議投資者在離新宿方圓五公里之內尋找穩定發展的區域，最好不要偏向東部的地區，因為那些區域比較靠海邊，風險較高。建議投資者投資已有租約10年以上的房產，而非租期較短的學生租戶，以減低風險。此外，文章也提到如果買的是套房，最好找穩定的家庭或上班族租戶，以減少空窗風險。
Summary of Segment-4:
文章描述了作者與一位師傅的對話，討論了在日本購房的注意事項和投資回報率。建議購買2LDK以上的房子，因為小套房需要支付較高的修繕費和管理費，而且租金可能會因遺留物和逃租造成損失。建議投資營業場合的房子，穩定且租約較長。日本的房子強度高，能活99年以上，越老的房子稅金越少。最後，該師傅提供了一個20平方公尺的房子作為投資的範例，每年稅金為6萬日元，但該房子陽台是免費的。
Summary of Segment-5:
這篇文章談到了作者曾買過一間10坪房子，雖然地點不錯但周遭有墓地，而討論了投資房地產的注意事項，如管理費用、修繕成本等。作者提到日本的房價雖然有漲但沒有像台灣那麼快，而且日本的年輕人不太買房子的原因與歷史背景有關，但現在日本房價回升，投資房地產時最好買人口集中地區且不要買套房，帶租約的營業場合比較穩定。最後作者分享了自己買過的一個風化區的房子，租金高達八萬，投報率達10%以上。
Summary of Segment-6:
這段文字描述了作者在日本購買房產時需

### Summarize All

In [21]:
# Put all together and send to chatgpt
all_res = ""
for res in result_arr:
    all_res+=res
final_summary = send_respond_gpt_turbo(all_res)
print(f"Total words = {len(final_summary)}")
print(final_summary)

Total words = 167
此文摘錄了幾篇文章，內容主要是關於在日本投資房地產的經驗分享和相關注意事項。文章提到在日本買房地產有風險，需要注意管理費、修繕費、稅金等，建議投資者要選擇穩定發展的區域和長租戶，並避免買套房等高風險物件。此外，文章也提到了作者買居酒屋的經驗和推薦商學院的播客節目。總而言之，投資房地產需要做好功課，選擇好物件，並仔細評估成本和回報率。
